# Simple FairGBM example on the UCI Adult dataset

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

In [2]:
SEED = 42

### Load data

In [ ]:
from utils import load_uci_adult, split_X_Y_S_uci_adult

train_set, test_set = load_uci_adult()
X_train, Y_train, S_train = split_X_Y_S_uci_adult(train_set)
X_test, Y_test, S_test = split_X_Y_S_uci_adult(test_set);

100% [..........................................................................] 3974305 / 3974305

###  Construct FairGBM model

In [ ]:
from fairgbm import FairGBMClassifier

core_lgbm_params = {
    "n_estimators": 200,
    "random_state": SEED,
    "n_jobs": -2,
}

# Instantiate
fairgbm_clf = FairGBMClassifier(
    constraint_type="FNR",    # constraint on equal group-wise TPR (equal opportunity)
    multiplier_learning_rate=0.2,
    constraint_fnr_slack=0.05,
    **core_lgbm_params,
)

### Train model

In [ ]:
%%time
# Train using features (X), labels (Y), and sensitive attributes (S)
fairgbm_clf.fit(X_train, Y_train, constraint_group=S_train)

### Compute binary predictions

In [ ]:
Y_test_pred = fairgbm_clf.predict(X_test)

### Compute Performance and Fairness metrics

In [ ]:
from sklearn.metrics import accuracy_score
from utils import compute_fairness_ratio

acc_val = accuracy_score(Y_test, Y_test_pred)
eq_op_val = compute_fairness_ratio(Y_test, Y_test_pred, S_test, "TPR")

In [ ]:
print(f"FairGBM:")
print(f"\tAccuracy: {acc_val:.1%}")
print(f"\tFairness: {eq_op_val:.1%}")

### Compare with vanilla LightGBM

In [ ]:
from fairgbm import LGBMClassifier

# Instantiate
lgbm_clf = LGBMClassifier(**core_lgbm_params)

# Train
lgbm_clf.fit(X_train, Y_train)

# Predict
Y_test_pred_lgbm = lgbm_clf.predict(X_test)

In [ ]:
lgbm_acc_val = accuracy_score(Y_test, Y_test_pred_lgbm)
lgbm_eq_op_val = compute_fairness_ratio(Y_test, Y_test_pred_lgbm, S_test, "TPR")

print(f"Vanilla LightGBM:")
print(f"\tAccuracy: {lgbm_acc_val:.1%}")
print(f"\tFairness: {lgbm_eq_op_val:.1%}")